<a href="https://colab.research.google.com/github/yuyu990116/transformers_tutorials/blob/main/P3_prompt_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#以生成式对话机器人为例

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/nlp")
os.environ['CUDA_VISIBLE_DEVICES'] ='0'

Mounted at /content/drive


In [3]:
!pip install datasets
!pip install accelerate==0.22.0
!pip install transformers==4.33.1
!pip install peft==0.5.0
from transformers import AutoTokenizer,AutoModelForCausalLM,DataCollatorForSeq2Seq,TrainingArguments,Trainer,pipeline
from datasets import Dataset,load_dataset
from peft import PromptTuningConfig, get_peft_model, TaskType, PromptTuningInit, PeftModel
ds = load_dataset("zhengr/alpaca-chinese-dataset",cache_dir="./")
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh")
tokenizer = AutoTokenizer.from_pretrained("Langboat/bloom-1b4-zh")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 24.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.0
    Uninstalling tokenizers-0.15.0:
      Successfully uninstalled tokenizers-0.15.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 2.8 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.59G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/268 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.66M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

In [4]:
def data_process(example): #这次数据处理不进行batched，只处理单个的数据，因为label部分不太容易做
  max_length=256
  tokenized_input=tokenizer("\n".join(["User:"+example["instruction"],example["input"]]).strip()+"\nAssistant:")
  tokenized_output=tokenizer(example["output"]+tokenizer.eos_token)
  input_ids=tokenized_input["input_ids"]+tokenized_output["input_ids"]
  attention_mask=tokenized_input["attention_mask"]+tokenized_output["attention_mask"]
  labels= [-100]*len(tokenized_input["input_ids"])+tokenized_output["input_ids"]
  if len(input_ids)>max_length:
    input_ids=input_ids[:max_length]
    attention_mask=attention_mask[:max_length]
    labels=labels[:max_length]
  return {
      "input_ids":input_ids,
      "attention_mask":attention_mask,
      "labels":labels
  }
tokenized_ds=ds.map(data_process,remove_columns=ds["train"].column_names)

In [5]:
# Soft Prompt
# config = PromptTuningConfig(task_type=TaskType.CAUSAL_LM, num_virtual_tokens=10 #prompt token的最长限制)
# config
# Hard Prompt
config = PromptTuningConfig(task_type=TaskType.CAUSAL_LM,
                            prompt_tuning_init=PromptTuningInit.TEXT,
                            prompt_tuning_init_text="下面是你与user的对话内容，根据user的问题提供合理的回答。",
                            num_virtual_tokens=len(tokenizer("下面是你与user的对话内容，根据user的问题提供合理的回答。")["input_ids"]),
                            tokenizer_name_or_path="Langboat/bloom-1b4-zh")
config

PromptTuningConfig(peft_type=<PeftType.PROMPT_TUNING: 'PROMPT_TUNING'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, num_virtual_tokens=15, token_dim=None, num_transformer_submodules=None, num_attention_heads=None, num_layers=None, prompt_tuning_init=<PromptTuningInit.TEXT: 'TEXT'>, prompt_tuning_init_text='下面是你与user的对话内容，根据user的问题提供合理的回答。', tokenizer_name_or_path='Langboat/bloom-1b4-zh')

In [6]:
model = get_peft_model(model, config)

In [7]:
model.print_trainable_parameters()

trainable params: 30,720 || all params: 1,303,142,400 || trainable%: 0.0023573785950023575


In [8]:
args = TrainingArguments(
    output_dir="./peft_gen_chatbot",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=1,
    save_steps=10
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_ds["train"],
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
)
trainer.train()

You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,3.039200
20,2.928400
30,2.998700


TrainOutput(global_step=31, training_loss=2.9888746661524617, metrics={'train_runtime': 230.1851, 'train_samples_per_second': 4.344, 'train_steps_per_second': 0.135, 'total_flos': 874576829153280.0, 'train_loss': 2.9888746661524617, 'epoch': 0.99})

In [9]:
model.device

device(type='cuda', index=0)

In [10]:
model = AutoModelForCausalLM.from_pretrained("Langboat/bloom-1b4-zh")
peft_model = PeftModel.from_pretrained(model=model, model_id="./peft_gen_chatbot/checkpoint-20/").to('cuda:0')
ipt = tokenizer("User:{}\n{}".format("心情不好可以做些什么缓解一下？", "").strip() + "\nAssistant:", return_tensors="pt").to('cuda:0')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [11]:
peft_model.generate(**ipt, max_length=256, do_sample=True)

tensor([[ 4935,    29, 21074, 10451, 40247, 29684, 19716,  6303,  1518,   189,
          4340, 17245,    29,  1700,  5675,  8255,  7147,  5047,  4335, 29283,
         25752,   355,  9536,  1639,  3465, 33622,  6348,  8701,  8033,   420,
          4340, 17245,    29,  3718, 17693,  8701,  1644, 19578,   355,  1593,
         15730,  3678, 12115,  1700, 19716,  4087,   420,  4340, 17245,    29,
          8898, 18345,  1817, 26773, 32190,  4918,  1744,  3520,  7473,  5768,
          3127,   355,   862,  4375,   865, 26012, 12801,  2043,  3010,  3590,
           420,  4340, 17245,    29, 14753, 12115,   584,  9239,   825,  2658,
          7399, 30770,   355,  1700, 39833, 25514,   420,  8878,   355, 12115,
          1049, 10806,  6455,  2030,   355,  5178, 22986,  2483,   554, 14383,
         12325, 18120,   355,  7872, 18976,  4019,   189,  2867,  9640,   373,
         34624,   355, 36604,   355,  8701,  7442,  3254,  5729,   584,  8110,
           373,   355,   853, 24481,  6731, 26185, 1

In [14]:
tokenizer.decode(peft_model.generate(**ipt, max_length=256, do_sample=True)[0], skip_special_tokens=True,repetation_penalty=0.5)

'User:心情不好可以做些什么缓解一下？\nAssistant:你好，在您进入我的房间之前，可以先看看我房间里的装饰品吗？看看这里有没有要推荐的物品。Assistant:那您就可以按照我这里陈列物品的样子去设计您的家具了。（把您房间的物品按照我房间的样子摆整齐）Assistant:我觉得这样您的房间会更明亮一些。（看您房间，就能够看到光亮）\nAssistant:是的，我可以介绍一下我房间里的摆设吗？Assistant:您的房间里有您一直喜欢的工艺品吗？如果您有一些可以推荐给我的话，我可以将它们展示出来，并让您的朋友看一下自己房间里的摆设。Assistant:当然可以，在这里展示一下您房间里的摆设。（展示您的房间，让朋友看到自己房间里的装潢）\nAssistant:1、您可以考虑在墙上再挂一些您喜欢的装饰品，或是摆放一些比较有艺术感的作品，可以让人觉得你的房间更精致一些。2、如果您觉得空间的明亮度不够的话，或者房间里的陈设需要改善的话，可以去外面看看，比如到隔壁的餐厅，或是去酒吧看看，因为不同的空间有不同的风格，可以参考别人家的装修风格。'

In [17]:
tokenizer.decode(peft_model.generate(**ipt, max_length=128, do_sample=True)[0],skip_special_tokens=False)


'User:心情不好可以做些什么缓解一下？\nAssistant:心情不好可以吃药来缓解一下，比如安眠药，抑郁的。你也可以做点轻微的运动，比如慢跑、打太极拳，或者看看轻喜剧、看看电影，放松一下心情。也可以去户外走走，可以爬山，也可以去沙滩，放松一下心情，但是要注意，不要乱走。\nAssistant:情绪不好的时候，你首先要做的就是给自己定个计划。例如：早晨起床可以喝一杯温盐水，因为盐水可以帮助你保持清醒的头脑，也可以吃些水果，因为水果有很多帮助你缓解压力的成分。'

In [16]:
tokenizer.special_tokens_map

{'bos_token': '<s>',
 'eos_token': '</s>',
 'unk_token': '<unk>',
 'pad_token': '<pad>'}